In [24]:
# COMP30027 Machine learning Project 1: Gaining Information about Naive Bayes
# Author: Jordan Ung <jordanu@student.unimelb.edu.au> [729938]
# Last Modified: 01.04.19

#### GENERAL GUIDE
# Read In And Inspect The Data
# Check for missing value - (1) delete rows with missing values, (2) Impute the missing values with dataset
# Check for anomalously extreme values

#TODO
# Predict is NOT working, Train() seems to be fine. Find out the problem

# Work on Info_Gain Function, why the fk is IG < 0
# NOTE: Entropy > 0 is fine, for x possible attribute values, the entropy is at max log2(x)

In [25]:
# Import dependencies
import math

In [26]:
# Preprocess takes the name of a file and returns a list of instances
# within that file, with each instance containing a list of attributes
def preprocess(file_name):
    dataset = []
    with open(file_name, 'r') as file:
        # Add each instance to a list to be used later
        for line in file.readlines():
            dataset.append(line.strip().split(','))
            
    # Group all the instances with the same class together
    dataset = sorted(dataset, key=lambda x: x[-1])
    return dataset

data_file = 'anneal.csv'
new_file = "_unit_test.csv"
missing_value_files = ['breast-cancer.csv', 'hepatitis.csv', 'hypothyroid.csv', 'mushroom.csv', 'primary-tumor.csv']

In [44]:
# Train takes a list of instances and returns a 3-tuple containing:
# A dictionary of the class distribution of all classes in the dataset
# A list of dictionaries, tallying each attribute value for every attribute
# A list (for each class) of dictionaries tallying attribute values for
# every attribute of a particular class
def train(instance_list, missing_value):
    data_info = ({}, [], [])

    current_class = instance_list[0][-1]
    data_info[0][current_class] = 0
    data_info[2].append([])
    class_index = 0
    
    # Add attribute lists to store the unique attribute values in
    for i in range(len(instance_list[0]) - 1):
        data_info[1].append({})
        data_info[2][class_index].append({})
    
    # Tally each value in each attribute for each class
    for data in instance_list:
#         print(data)
        attribute_num = 0

        # New class has been detected
        if data[-1] != current_class:

            # Add data structure to support new class
            current_class = data[-1]
            data_info[0][current_class] = 0
            data_info[2].append([])
            class_index += 1
            
            # Add dictionary for each attribute
            for i in range(len(data) - 1):
                data_info[2][class_index].append({})
        
        # Input each instance's attribute into the appropriate dictionary
        for attribute in data[:-1]:
            
            if attribute in data_info[1][attribute_num]:
                data_info[1][attribute_num][attribute] += 1
            else:
                data_info[1][attribute_num][attribute] = 1
                
            if attribute in data_info[2][class_index][attribute_num]:
                data_info[2][class_index][attribute_num][attribute] += 1
            else:
                data_info[2][class_index][attribute_num][attribute] = 1
            attribute_num += 1
        
        data_info[0][current_class] += 1
        
    return data_info # Return the 3-tuple


# train(preprocess(missing_value_files[0]), "?")
train(preprocess(new_file), "?")

# train(preprocess(missing_value_files[-1]), "?")

({'cold': 2, 'flu': 3},
 [{'mild': 2, 'no': 1, 'severe': 2},
  {'mild': 2, 'no': 1, 'severe': 2},
  {'high': 1, 'normal': 4},
  {'no': 1, 'yes': 4}],
 [[{'mild': 1, 'no': 1},
   {'no': 1, 'severe': 1},
   {'normal': 2},
   {'no': 1, 'yes': 1}],
  [{'mild': 1, 'severe': 2},
   {'mild': 2, 'severe': 1},
   {'high': 1, 'normal': 2},
   {'yes': 3}]])

In [33]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instances):
    predicted_classes = []
    
    
    # Find class prediction for each class
    for data in instances:
        class_index = 0
        attribute_index = 0
        probability_of_class = 0
        
        # Calculate probability of the instance belonging to a particular class
        for class_name in model[0].keys():
            
            print(class_name)
        
#         attribute_number = 0
#         probability_of_class = []
        
#         # Find probability of an instance being of a particular class
#         for class_index in range(len(model[2])):
#             probability = 1.0
#             for attribute in model[2][class_index]:
#                 if data[attribute_number] in attribute:
#                     probability *= attribute[data[attribute_number]] / model[1][class_index]
#                 else:
#                     probability /= len(model[2][class_index])
#             probability_of_class.append(probability)
        
#         class_index = 0
#         highest_probability = 0
#         # Find the class with the highest probability
#         for i in range(len(probability_of_class)):
#             if probability_of_class[i] > highest_probability:
#                 highest_probability = probability_of_class[i]
#                 class_index = i

# #                 print("Higher")
# #             elif probability_of_class[i] < highest_probability:
# #                 print("Less Than")
# #             else:
# #                 print("Equal Too")
#         print(data, "has predicted value", model[0][class_index])
#         predicted_classes.append(model[0][class_index])
#         print("--------------------")

#     #### NAIVE WAY OF PUTTING HIGHEST CLASS AS ALL PREDICTIONS
#     top_class = ""
#     highest = 0
#     for i in model[0]:
#         if model[0][i] > highest:
#             top_class = i
#             highest = model[0][i]
    
#     for j in range(len(instances)):
#         predicted_classes.append(top_class)
        
    return predicted_classes # Return a list of predicted classes

predict((train(preprocess(new_file), "?")), preprocess(new_file))

# predict(train(preprocess(missing_value_files[1]), "?"), preprocess(missing_value_files[1]))

cold
flu
cold
flu
cold
flu
cold
flu
cold
flu


[]

In [32]:
# Evaluates the performance of the predictor model
def evaluate(predictions, dataset):
    tries = 0
    correct = 0
    
    for i in range(len(predictions)):
        if predictions[i] == dataset[i][-1]:
            correct += 1
        tries += 1
        
    print("Correct:", correct, "out of", tries)
    print("Accuracy Rate (%): ", correct / tries * 100)
    print("-----------------------------------")
    return

# print("Preprocess")
# print(preprocess(new_file))
# print("---------------")
# print("Train")
# print((train(preprocess(new_file), "?")))
# print("---------------")
# print(predict((train(preprocess(new_file), "?")), preprocess(new_file)))
# print("---------------")


# for i in missing_value_files:
#     evaluate(predict(train(preprocess(i), "?"), preprocess(i)), preprocess(i))


In [7]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(model):
    attribute_info_gain = []
    # Calculate the Information Gain of each attribute
    for attribute_num in range(len(model[2][0])):
        attribute_totals = []
        total_value = {}
        
        # Find all the unique values for a given attribute
        for class_index in range(len(model[0])):
            data = model[2][class_index][attribute_num]
            
            # Adds the class' attribute dictionary to the cumulative dictionary
            total_value = {x: total_value.get(x, 0) + data.get(x, 0) for x in set(total_value).union(data)}

        # Delete the 'missing values' totals
        if "?" in total_value:
            del total_value["?"]
        unique_values = set(total_value.keys())
            
        mean_info = 0
#         print("---------------")
#         print(total_value)
#         print(sum(total_value.values()))
#         print("---------------")
        
        attribute_entropy = 0
        for i in total_value:
            attribute_probability = total_value[i] * 1.0 / sum(total_value.values())
            attribute_entropy -= attribute_probability * math.log2(attribute_probability)
        
        # Calculate entropy for all different values within an attribute
        for value in unique_values:
            entropy = 0
            for class_index in range(len(model[0])):
                data = model[2][class_index][attribute_num]
#                 print(data)
                if value in data:
                    print("---", data[value], "---", total_value[value])
                    data_probability = data[value] / total_value[value]
                    entropy -= (data_probability * math.log2(data_probability))
                    print("Entropy ---", entropy, "---")
            print("Final Entropy:", entropy)
            mean_info += total_value[value] * 1.0 / sum(total_value.values()) * entropy
            
#         print("Mean Info", mean_info)
#         print("Attribute Entropy", attribute_entropy)
# #         print("---------------")
        print("ATTR", attribute_entropy - mean_info)
        # At this point: Found the Mean Info for an attribute
        
        # Find entropy of the attribute itself
        

    return attribute_info_gain
info_gain(train(preprocess(missing_value_files[0]), "?"))

--- 1 --- 1
Entropy --- 0.0 ---
Final Entropy: 0.0
--- 5 --- 6
Entropy --- 0.21919533819482817 ---
--- 1 --- 6
Entropy --- 0.6500224216483541 ---
Final Entropy: 0.6500224216483541
--- 63 --- 90
Entropy --- 0.3602012209808308 ---
--- 27 --- 90
Entropy --- 0.8812908992306927 ---
Final Entropy: 0.8812908992306927
--- 40 --- 57
Entropy --- 0.35856976791395045 ---
--- 17 --- 57
Entropy --- 0.8791357668533336 ---
Final Entropy: 0.8791357668533336
--- 21 --- 36
Entropy --- 0.45360442088707204 ---
--- 15 --- 36
Entropy --- 0.9798687566511528 ---
Final Entropy: 0.9798687566511528
--- 71 --- 96
Entropy --- 0.3218780423580173 ---
--- 25 --- 96
Entropy --- 0.8273744775003172 ---
Final Entropy: 0.8273744775003172
ATTR 1.171915868759835
--- 5 --- 7
Entropy --- 0.3467334479787441 ---
--- 2 --- 7
Entropy --- 0.863120568566631 ---
Final Entropy: 0.863120568566631
--- 102 --- 150
Entropy --- 0.37834747699658194 ---
--- 48 --- 150
Entropy --- 0.9043814577244937 ---
Final Entropy: 0.9043814577244937
--- 9

[]

In [8]:
print(-21.0/36 * math.log2(21.0/36) - 15.0/36 * math.log2(15.0/36))

0.9798687566511528


In [46]:
a = {}
a["word"] = {}
print(a)

{'word': {}}
